In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageEnhance, ImageChops
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim
from diffusers import StableDiffusionInpaintPipeline
import cv2

# -------------------------- DEVICE SETUP --------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -------------------------- DATASET --------------------------
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = [
            os.path.join(root_dir, f)
            for f in os.listdir(root_dir)
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img

# -------------------------- U-NET MODEL --------------------------
class UNet(nn.Module):
    def __init__(self, in_channels=6, out_channels=3):
        super(UNet, self).__init__()
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        self.pool = nn.MaxPool2d(2, 2)
        self.bottleneck = self.conv_block(512, 1024)
        self.upconv4 = self.upconv_block(1024, 512)
        self.dec4 = self.conv_block(1024, 512)
        self.upconv3 = self.upconv_block(512, 256)
        self.dec3 = self.conv_block(512, 256)
        self.upconv2 = self.upconv_block(256, 128)
        self.dec2 = self.conv_block(256, 128)
        self.upconv1 = self.upconv_block(128, 64)
        self.dec1 = self.conv_block(128, 64)
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True)
        )

    def upconv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        bottleneck = self.bottleneck(self.pool(enc4))
        dec4 = self.upconv4(bottleneck)
        enc4_resized = F.interpolate(enc4, size=dec4.shape[2:], mode="bilinear", align_corners=False)
        dec4 = torch.cat([dec4, enc4_resized], dim=1)
        dec4 = self.dec4(dec4)
        dec3 = self.upconv3(dec4)
        enc3_resized = F.interpolate(enc3, size=dec3.shape[2:], mode="bilinear", align_corners=False)
        dec3 = torch.cat([dec3, enc3_resized], dim=1)
        dec3 = self.dec3(dec3)
        dec2 = self.upconv2(dec3)
        enc2_resized = F.interpolate(enc2, size=dec2.shape[2:], mode="bilinear", align_corners=False)
        dec2 = torch.cat([dec2, enc2_resized], dim=1)
        dec2 = self.dec2(dec2)
        dec1 = self.upconv1(dec2)
        enc1_resized = F.interpolate(enc1, size=dec1.shape[2:], mode="bilinear", align_corners=False)
        dec1 = torch.cat([dec1, enc1_resized], dim=1)
        dec1 = self.dec1(dec1)
        return self.final_conv(dec1)

# -------------------------- ENHANCED ELA TAMPER DETECTION --------------------------
def perform_enhanced_ela(image_path, qualities=[90, 85, 75], scales=[10, 15, 20]):
    """
    Performs Enhanced Error Level Analysis on an image to detect tampering
    
    Args:
        image_path: Path to the image file or PIL Image object
        qualities: List of JPEG compression qualities for multi-level ELA
        scales: List of amplification factors for the ELA results
    
    Returns:
        PIL Image showing best ELA result, tampered mask, detected regions
    """
    if isinstance(image_path, Image.Image):
        original = image_path
    else:
        original = Image.open(image_path)
    
    original = original.convert("RGB")
    temp_file = "temp_ela.jpg"
    
    # Perform ELA at multiple quality levels
    ela_results = []
    ela_arrays = []
    
    for quality, scale in zip(qualities, scales):
        # Save with specific quality
        original.save(temp_file, "JPEG", quality=quality)
        
        # Open the saved file
        compressed = Image.open(temp_file)
        
        # Calculate the difference
        ela_image = ImageChops.difference(original, compressed)
        
        # Scale the difference for better visualization
        extrema = ela_image.getextrema()
        max_diff = max([ex[1] for ex in extrema])
        if max_diff == 0:
            max_diff = 1
        
        # Amplify the difference
        ela_image = ImageEnhance.Brightness(ela_image).enhance(scale / max_diff)
        ela_results.append(ela_image)
        ela_arrays.append(np.array(ela_image))
    
    # Combine ELA results for best detection
    combined_ela = np.maximum.reduce(ela_arrays)
    
    # Convert to image for visualization
    best_ela_image = Image.fromarray(combined_ela.astype('uint8'))
    
    # Process for mask creation
    gray_ela = cv2.cvtColor(combined_ela, cv2.COLOR_RGB2GRAY)
    
    # Apply bilateral filter to preserve edges while smoothing
    gray_ela_filtered = cv2.bilateralFilter(gray_ela, 9, 75, 75)
    
    # Use adaptive thresholding to identify tampering
    # Calculate dynamic threshold based on image statistics
    mean_intensity = np.mean(gray_ela_filtered)
    std_intensity = np.std(gray_ela_filtered)
    threshold_value = mean_intensity + 2 * std_intensity
    
    _, binary_mask = cv2.threshold(
        gray_ela_filtered, 
        threshold_value, 
        255, 
        cv2.THRESH_BINARY
    )
    
    # Enhance binary mask using local adaptive thresholding
    adaptive_mask = cv2.adaptiveThreshold(
        gray_ela_filtered,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        11,
        2
    )
    
    # Combine both masks for better results
    combined_mask = cv2.bitwise_or(binary_mask, adaptive_mask)
    
    # Clean up the mask with morphological operations
    kernel = np.ones((5, 5), np.uint8)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    
    # Find contours for region analysis
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter contours by size to remove noise
    min_area = original.size[0] * original.size[1] * 0.005  # 0.5% of image area
    significant_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]
    
    # Create a mask with only significant regions
    refined_mask = np.zeros_like(combined_mask)
    cv2.drawContours(refined_mask, significant_contours, -1, 255, -1)
    
    # Detect tampering regions and highlight in original image
    detection_overlay = np.array(original)
    for contour in significant_contours:
        # Draw bounding box around suspicious regions
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(detection_overlay, (x, y), (x + w, y + h), (0, 0, 255), 2)
    
    # Remove the temporary file
    if os.path.exists(temp_file):
        os.remove(temp_file)
    
    return best_ela_image, refined_mask, Image.fromarray(detection_overlay)

def is_image_tampered(ela_data, mask=None, threshold_percentage=1.0):
    """
    Determine if an image is tampered based on ELA results
    
    Args:
        ela_data: The ELA image result or mask
        mask: Optional binary mask from ELA
        threshold_percentage: The threshold percentage of image coverage to determine tampering
    
    Returns:
        Boolean indicating if tampering is detected, tamper score
    """
    # If mask is provided, use it for detection
    if mask is not None:
        if isinstance(mask, Image.Image):
            mask_array = np.array(mask)
        else:
            mask_array = mask
            
        # Calculate the percentage of image covered by suspected tampering
        coverage = np.sum(mask_array > 0) / mask_array.size * 100
        
        # Calculate tamper score based on size and intensity of tampering regions
        # Higher values indicate higher likelihood of tampering
        if np.max(mask_array) > 0:
            mean_intensity = np.mean(mask_array[mask_array > 0])
        else:
            mean_intensity = 0
            
        tamper_score = coverage * (mean_intensity / 255)
        
        # Check if the coverage exceeds the threshold
        is_tampered = coverage > threshold_percentage
        return is_tampered, tamper_score
    
    # If mask not provided, analyze ELA directly
    if isinstance(ela_data, Image.Image):
        ela_array = np.array(ela_data)
    else:
        ela_array = ela_data
    
    # Calculate metrics from ELA result
    gray_ela = cv2.cvtColor(ela_array, cv2.COLOR_RGB2GRAY) if len(ela_array.shape) > 2 else ela_array
    
    # Calculate local variance map to detect irregularities
    local_var = cv2.Laplacian(gray_ela, cv2.CV_64F).var()
    
    # Calculate mean intensity of the ELA result
    mean_intensity = np.mean(gray_ela)
    
    # Calculate histogram features
    hist = cv2.calcHist([gray_ela], [0], None, [256], [0, 256])
    hist_norm = hist / hist.sum()
    hist_entropy = -np.sum(hist_norm * np.log2(hist_norm + 1e-7))
    
    # Combined tamper score 
    tamper_score = (mean_intensity / 255) * 0.3 + (local_var / 1000) * 0.4 + (hist_entropy / 8) * 0.3
    
    # Determine if tampered based on score
    is_tampered = tamper_score > 0.15
    
    return is_tampered, tamper_score

# -------------------------- UTILS --------------------------
def create_random_tampered_mask(image_size=(512, 512), min_size=100, max_size=300, num_patches=3):
    mask = np.zeros(image_size, dtype=np.uint8)
    for _ in range(num_patches):  # Multiple patches for more significant tampering
        w, h = np.random.randint(min_size, max_size, size=2)
        x1 = np.random.randint(0, image_size[0] - w)
        y1 = np.random.randint(0, image_size[1] - h)
        x2 = x1 + w
        y2 = y1 + h
        mask[y1:y2, x1:x2] = 255
    return mask

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting", 
    safety_checker=None
).to("cuda")

def generate_tampered_image(image, mask):
    result = pipe(
        prompt="a realistic filling for tampered area", 
        image=image,
        mask_image=mask,
        num_inference_steps=5,
        guidance_scale=3
    ).images[0]
    return result

def enhance_image(image):
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(1.2)
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.3)
    enhancer = ImageEnhance.Sharpness(image)
    image = enhancer.enhance(2.0)
    return image

def recover_image_with_unet(tampered_image, original_image):
    input_image = torch.cat([tampered_image, original_image], dim=1).requires_grad_()
    recovered_image = unet_model(input_image)
    return recovered_image

def immunize_image(original_image):
    dummy_tampered = original_image.clone()
    input_image = torch.cat([dummy_tampered, original_image], dim=1).requires_grad_()
    immunized = unet_model(input_image)
    return immunized

def calculate_metrics(original, target):
    # Check if original is a batch of images (4D tensor) and take the first one if so
    if original.dim() == 4 and original.size(0) == 1:
        original = original.squeeze(0)
    
    # Check if target is a batch of images (4D tensor) and take the first one if so
    if target.dim() == 4 and target.size(0) == 1:
        target = target.squeeze(0)
    
    # Convert to numpy arrays via PIL
    original_np = np.array(transforms.ToPILImage()(original.cpu()))
    target_np = np.array(transforms.ToPILImage()(target.cpu()))
    
    # Calculate metrics
    psnr = compare_psnr(original_np, target_np, data_range=255)
    ssim = compare_ssim(original_np, target_np, channel_axis=-1, data_range=255)
    
    return psnr, ssim

# -------------------------- PATHS --------------------------
OUTPUT_IMMUNIZED = "/kaggle/working/immunized"
OUTPUT_FORGERY = "/kaggle/working/forgery"
OUTPUT_RECOVERED = "/kaggle/working/recovered"
OUTPUT_ELA = "/kaggle/working/ela"
OUTPUT_IMMUNIZED_TAMPERED = "/kaggle/working/immunized_tampered"
OUTPUT_IMMUNIZED_RECOVERED = "/kaggle/working/immunized_recovered"
OUTPUT_IMMUNIZED_ELA = "/kaggle/working/immunized_ela"
os.makedirs(OUTPUT_IMMUNIZED, exist_ok=True)
os.makedirs(OUTPUT_FORGERY, exist_ok=True)
os.makedirs(OUTPUT_RECOVERED, exist_ok=True)
os.makedirs(OUTPUT_ELA, exist_ok=True)
os.makedirs(OUTPUT_IMMUNIZED_TAMPERED, exist_ok=True)
os.makedirs(OUTPUT_IMMUNIZED_RECOVERED, exist_ok=True)
os.makedirs(OUTPUT_IMMUNIZED_ELA, exist_ok=True)

# -------------------------- DATA LOADING --------------------------
DATA_PATH = "/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Au"
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])
dataset = CustomImageDataset(root_dir=DATA_PATH, transform=transform)
num_images_to_process = 100
limited_dataset = torch.utils.data.Subset(dataset, range(num_images_to_process))
dataloader = DataLoader(limited_dataset, batch_size=2, shuffle=True, num_workers=2, pin_memory=True)


# -------------------------- MODEL INIT --------------------------
unet_model = UNet(in_channels=6).to(device)
unet_criterion = nn.MSELoss()
unet_optimizer = optim.Adam(unet_model.parameters(), lr=1e-4)

# -------------------------- TRAINING --------------------------
EPOCHS = 5
max_iterations_per_epoch = 100

for epoch in range(EPOCHS):
    unet_model.train()
    running_loss = 0.0
    for i, images in enumerate(tqdm(dataloader)):
        if i >= max_iterations_per_epoch:
            break
        images = images.to(device).requires_grad_()
        original_images = images.clone()
        tampered_images = []
        for i in range(images.size(0)):
            img = transforms.ToPILImage()(images[i][:3].cpu())
            img_resized = img.resize((512, 512))
            mask = create_random_tampered_mask(image_size=(512, 512))
            mask_pil = Image.fromarray(mask).convert("L")
            result = generate_tampered_image(img_resized, mask_pil)
            result = enhance_image(result)
            result = result.resize((512, 512))
            tampered_images.append(transforms.ToTensor()(result).unsqueeze(0))
            del img, img_resized, result
        tampered_images = torch.cat(tampered_images).to(device)
        recovered_images = recover_image_with_unet(tampered_images, original_images)
        loss = unet_criterion(recovered_images, original_images)
        unet_optimizer.zero_grad()
        loss.backward()
        unet_optimizer.step()
        running_loss += loss.item()
        del tampered_images, recovered_images
        torch.cuda.empty_cache()
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(dataloader):.6f}")

print("✅ U-Net Training Completed!")

# -------------------------- GENERATE RESULTS --------------------------
def visualize_results(original_img, tampered_img, recovered_img, immunized_img, immunized_tampered_img, immunized_recovered_img, ela_img, immunized_ela_img, mask_img, immunized_mask_img, detection_img, immunized_detection_img, idx, i, is_tampered, immunized_is_tampered, tamper_score, immunized_tamper_score):
    # First figure - Original flow
    fig1, axes1 = plt.subplots(2, 4, figsize=(20, 12))
    
    # Original image
    axes1[0, 0].imshow(original_img)
    axes1[0, 0].set_title(f"Original")
    axes1[0, 0].axis("off")
    
    # Tampered image
    axes1[0, 1].imshow(tampered_img)
    axes1[0, 1].set_title(f"Tampered")
    axes1[0, 1].axis("off")
    
    # ELA result
    axes1[0, 2].imshow(ela_img)
    axes1[0, 2].set_title(f"ELA Result\nDetected as {'Tampered' if is_tampered else 'Authentic'}\nScore: {tamper_score:.4f}")
    axes1[0, 2].axis("off")
    
    # Detection visualization
    axes1[0, 3].imshow(detection_img)
    axes1[0, 3].set_title("Tampering Detection")
    axes1[0, 3].axis("off")
    
    # Tamper mask
    axes1[1, 0].imshow(mask_img, cmap='gray')
    axes1[1, 0].set_title("Tampering Mask")
    axes1[1, 0].axis("off")
    
    # Recovered image
    axes1[1, 1].imshow(recovered_img)
    axes1[1, 1].set_title("Recovered")
    axes1[1, 1].axis("off")
    
    # Immunized image
    axes1[1, 2].imshow(immunized_img)
    axes1[1, 2].set_title("Immunized")
    axes1[1, 2].axis("off")
    
    # Difference map between original and tampered
    if isinstance(original_img, np.ndarray):
        diff_map = cv2.absdiff(np.array(original_img), np.array(tampered_img))
    else:
        original_np = np.array(original_img)
        tampered_np = np.array(tampered_img)
        diff_map = cv2.absdiff(original_np, tampered_np)
    axes1[1, 3].imshow(diff_map)
    axes1[1, 3].set_title("Difference Map")
    axes1[1, 3].axis("off")
    
    plt.tight_layout()
    # Create the directory if it doesn't exist
    os.makedirs("/kaggle/working/outputs", exist_ok=True)

# Now you can save the figure
    plt.savefig(f"/kaggle/working/outputs/results_{idx}_{i}.png")
    plt.close()
    
    # Second figure - Immunized flow 
    fig2, axes2 = plt.subplots(2, 4, figsize=(20, 12))
    
    # Immunized image
    axes2[0, 0].imshow(immunized_img)
    axes2[0, 0].set_title(f"Immunized")
    axes2[0, 0].axis("off")
    
    # Tampered immunized image
    axes2[0, 1].imshow(immunized_tampered_img)
    axes2[0, 1].set_title(f"Tampered Immunized")
    axes2[0, 1].axis("off")
    
    # ELA result on immunized
    axes2[0, 2].imshow(immunized_ela_img)
    axes2[0, 2].set_title(f"Immunized ELA\nDetected as {'Tampered' if immunized_is_tampered else 'Authentic'}\nScore: {immunized_tamper_score:.4f}")
    axes2[0, 2].axis("off")
    
    # Detection visualization
    axes2[0, 3].imshow(immunized_detection_img)
    axes2[0, 3].set_title("Immunized Tampering Detection")
    axes2[0, 3].axis("off")
    
    # Tamper mask for immunized
    axes2[1, 0].imshow(immunized_mask_img, cmap='gray')
    axes2[1, 0].set_title("Immunized Tampering Mask")
    axes2[1, 0].axis("off")
    
    # Recovered immunized image
    axes2[1, 1].imshow(immunized_recovered_img)
    axes2[1, 1].set_title("Recovered Immunized")
    axes2[1, 1].axis("off")
    
    # Empty space or additional visualization
    if isinstance(immunized_img, np.ndarray):
        immunized_diff_map = cv2.absdiff(np.array(immunized_img), np.array(immunized_tampered_img))
    else:
        immunized_np = np.array(immunized_img)
        immunized_tampered_np = np.array(immunized_tampered_img)
        immunized_diff_map = cv2.absdiff(immunized_np, immunized_tampered_np)
    axes2[1, 2].imshow(immunized_diff_map)
    axes2[1, 2].set_title("Immunized Difference Map")
    axes2[1, 2].axis("off")
    
    # Compare original and immunized
    if isinstance(original_img, np.ndarray):
        compare_diff = cv2.absdiff(np.array(original_img), np.array(immunized_img))
    else:
        original_np = np.array(original_img)
        immunized_np = np.array(immunized_img)
        compare_diff = cv2.absdiff(original_np, immunized_np)
    axes2[1, 3].imshow(compare_diff)
    axes2[1, 3].set_title("Original vs Immunized")
    axes2[1, 3].axis("off")
    
    plt.tight_layout()
    plt.savefig(f"outputs/results_immunized_{idx}_{i}.png")
    plt.close()

unet_model.eval()
for idx, images in enumerate(dataloader):
    if idx >= 5:  # Process only a few batches for demo
        break
    
    images = images.to(device)
    original_images = images.clone()
    
    # Process each image in the batch
    for i in range(images.size(0)):
        original_img = transforms.ToPILImage()(original_images[i][:3].cpu())
        
        # -------------------------- ORIGINAL IMAGE FLOW --------------------------
        # Create tampered version of original
        img_resized = original_img.resize((512, 512))
        mask = create_random_tampered_mask(image_size=(512, 512))
        mask_pil = Image.fromarray(mask).convert("L")
        tampered_img = generate_tampered_image(img_resized, mask_pil)
        tampered_img = enhance_image(tampered_img)
        tampered_img = tampered_img.resize((512, 512))
        
        # Save JPG version for ELA
        tampered_jpg_path = os.path.join(OUTPUT_FORGERY, f"tampered_{idx}_{i}.jpg")
        tampered_img.save(tampered_jpg_path, "JPEG", quality=95)
        
        # Perform Enhanced ELA on the tampered image
        ela_img, tamper_mask, detection_img = perform_enhanced_ela(tampered_jpg_path)
        ela_img.save(os.path.join(OUTPUT_ELA, f"ela_{idx}_{i}.png"))
        
        # Determine if image is tampered using ELA
        is_tampered, tamper_score = is_image_tampered(ela_img, tamper_mask)
        
        # Convert to tensor for recovery
        tampered_tensor = transforms.ToTensor()(tampered_img).unsqueeze(0).to(device)
        
        with torch.no_grad():
            # Combine tampered and original for recovery
            input_tensor = torch.cat([tampered_tensor, original_images[i].unsqueeze(0)], dim=1)
            recovered_tensor = unet_model(input_tensor)
            
            # Generate immunized version
            dummy_tampered = original_images[i].unsqueeze(0).clone()
            input_tensor = torch.cat([dummy_tampered, original_images[i].unsqueeze(0)], dim=1)
            immunized_tensor = unet_model(input_tensor)
        
        # Convert tensors back to images
        recovered_img = transforms.ToPILImage()(recovered_tensor[0].cpu())
        immunized_img = transforms.ToPILImage()(immunized_tensor[0].cpu())
        
        # Save results
        recovered_img.save(os.path.join(OUTPUT_RECOVERED, f"recovered_{idx}_{i}.png"))
        immunized_img.save(os.path.join(OUTPUT_IMMUNIZED, f"immunized_{idx}_{i}.png"))
        
        # -------------------------- IMMUNIZED IMAGE FLOW --------------------------
        # Now apply the same tampering process to the immunized image
        immunized_img_resized = immunized_img.resize((512, 512))
        
        # Use the same mask for consistency in comparison
        immunized_tampered_img = generate_tampered_image(immunized_img_resized, mask_pil)
        immunized_tampered_img = enhance_image(immunized_tampered_img)
        immunized_tampered_img = immunized_tampered_img.resize((512, 512))
        
        # Save JPG version for ELA
        immunized_tampered_jpg_path = os.path.join(OUTPUT_IMMUNIZED_TAMPERED, f"immunized_tampered_{idx}_{i}.jpg")
        immunized_tampered_img.save(immunized_tampered_jpg_path, "JPEG", quality=95)
        
        # Perform Enhanced ELA on the tampered immunized image
        immunized_ela_img, immunized_tamper_mask, immunized_detection_img = perform_enhanced_ela(immunized_tampered_jpg_path)
        immunized_ela_img.save(os.path.join(OUTPUT_IMMUNIZED_ELA, f"immunized_ela_{idx}_{i}.png"))
        
        # Determine if immunized image is tampered using ELA
        immunized_is_tampered, immunized_tamper_score = is_image_tampered(immunized_ela_img, immunized_tamper_mask)
        
        # Convert tampered immunized image to tensor for recovery
        immunized_tampered_tensor = transforms.ToTensor()(immunized_tampered_img).unsqueeze(0).to(device)
        
        with torch.no_grad():
            # Recover the tampered immunized image
            # Use the original image as reference for recovery
            immunized_input_tensor = torch.cat([immunized_tampered_tensor, original_images[i].unsqueeze(0)], dim=1)
            immunized_recovered_tensor = unet_model(immunized_input_tensor)
        
        # Convert tensor back to image
        immunized_recovered_img = transforms.ToPILImage()(immunized_recovered_tensor[0].cpu())
        
        # Save results
        immunized_tampered_img.save(os.path.join(OUTPUT_IMMUNIZED_TAMPERED, f"immunized_tampered_{idx}_{i}.png"))
        immunized_recovered_img.save(os.path.join(OUTPUT_IMMUNIZED_RECOVERED, f"immunized_recovered_{idx}_{i}.png"))
        
        # Calculate metrics for original flow
        psnr_tampered, ssim_tampered = calculate_metrics(
            original_images[i][:3].unsqueeze(0), 
            tampered_tensor
        )
        psnr_recovered, ssim_recovered = calculate_metrics(
            original_images[i][:3].unsqueeze(0), 
            recovered_tensor
        )
        
        # Calculate metrics for immunized flow
        psnr_immunized, ssim_immunized = calculate_metrics(
            original_images[i][:3].unsqueeze(0), 
            immunized_tensor
        )
        psnr_immunized_tampered, ssim_immunized_tampered = calculate_metrics(
            immunized_tensor, 
            immunized_tampered_tensor
        )
        psnr_immunized_recovered, ssim_immunized_recovered = calculate_metrics(
            immunized_tensor, 
            immunized_recovered_tensor
        )
        
        print(f"\n📊 Image {idx}_{i}:")
        print(f"  ➤ ELA Detection: {'TAMPERED' if is_tampered else 'AUTHENTIC'} (Score: {tamper_score:.4f})")
        print(f"  ➤ PSNR Tampered   = {psnr_tampered:.2f}, SSIM Tampered   = {ssim_tampered:.4f}")
        print(f"  ➤ PSNR Recovered  = {psnr_recovered:.2f}, SSIM Recovered  = {ssim_recovered:.4f}")
        print(f"\n  ➤ Immunized Flow:")
        print(f"  ➤ ELA Detection on Immunized: {'TAMPERED' if immunized_is_tampered else 'AUTHENTIC'} (Score: {immunized_tamper_score:.4f})")
        print(f"  ➤ PSNR Immunized      = {psnr_immunized:.2f}, SSIM Immunized      = {ssim_immunized:.4f}")
        print(f"  ➤ PSNR Imm Tampered   = {psnr_immunized_tampered:.2f}, SSIM Imm Tampered   = {ssim_immunized_tampered:.4f}")
        print(f"  ➤ PSNR Imm Recovered  = {psnr_immunized_recovered:.2f}, SSIM Imm Recovered  = {ssim_immunized_recovered:.4f}")
        
        # Visualize and save results
        visualize_results(
            original_img, 
            tampered_img, 
            recovered_img, 
            immunized_img, 
            immunized_tampered_img,
            immunized_recovered_img,
            ela_img, 
            immunized_ela_img,
            Image.fromarray(tamper_mask), 
            Image.fromarray(immunized_tamper_mask),
            detection_img,
            immunized_detection_img,
            idx, 
            i, 
            is_tampered,
            immunized_is_tampered,
            tamper_score,
            immunized_tamper_score
        )

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Using device: cuda


model_index.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]


📊 Image 4_0:
  ➤ ELA Detection: TAMPERED (Score: 100.0000)
  ➤ PSNR Tampered   = 9.51, SSIM Tampered   = 0.4841
  ➤ PSNR Recovered  = 24.50, SSIM Recovered  = 0.9105

  ➤ Immunized Flow:
  ➤ ELA Detection on Immunized: TAMPERED (Score: 100.0000)
  ➤ PSNR Immunized      = 24.86, SSIM Immunized      = 0.9536
  ➤ PSNR Imm Tampered   = 10.88, SSIM Imm Tampered   = 0.3759
  ➤ PSNR Imm Recovered  = 26.76, SSIM Imm Recovered  = 0.9557


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]


📊 Image 4_1:
  ➤ ELA Detection: TAMPERED (Score: 100.0000)
  ➤ PSNR Tampered   = 11.41, SSIM Tampered   = 0.3336
  ➤ PSNR Recovered  = 23.21, SSIM Recovered  = 0.9211

  ➤ Immunized Flow:
  ➤ ELA Detection on Immunized: TAMPERED (Score: 100.0000)
  ➤ PSNR Immunized      = 27.42, SSIM Immunized      = 0.9525
  ➤ PSNR Imm Tampered   = 9.24, SSIM Imm Tampered   = 0.3383
  ➤ PSNR Imm Recovered  = 25.98, SSIM Imm Recovered  = 0.9465
